In [11]:
pip install tzlocal

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd

# for scraping app info from App Store
from itunes_app_scraper.scraper import AppStoreScraper

# for scraping app reviews from App Store
from app_store_scraper import AppStore

# for pretty printing data structures
from pprint import pprint

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time
import re

# for coupang-eats-rider & store
import xmltodict
from urllib.request import urlopen
import json

In [14]:
# 크롤링 진행할 때마다 fetched 되는 파일이 다름
## Instantiate AppStore class for desired app
my_app = AppStore(
  country='kr',        # required, 2-letter code
  app_name='coupang-eats', # required, found in app's url
  app_id=1445504255   # technically not required, found in app's url
) 
    
## Use review method to scrape reviews from App Store
my_app.review()  # if not provided, defaults to scraping all

2021-08-26 22:59:39,548 [INFO] Base - Initialised: AppStore('kr', 'coupang-eats', 1445504255)
2021-08-26 22:59:39,549 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/coupang-eats/id1445504255
2021-08-26 22:59:44,677 [INFO] Base - [id:1445504255] Fetched 340 reviews (340 fetched in total)
2021-08-26 22:59:50,322 [INFO] Base - [id:1445504255] Fetched 780 reviews (780 fetched in total)
2021-08-26 22:59:55,706 [INFO] Base - [id:1445504255] Fetched 1180 reviews (1180 fetched in total)
2021-08-26 23:00:01,329 [INFO] Base - [id:1445504255] Fetched 1480 reviews (1480 fetched in total)
2021-08-26 23:00:06,834 [INFO] Base - [id:1445504255] Fetched 1680 reviews (1680 fetched in total)
2021-08-26 23:00:12,226 [INFO] Base - [id:1445504255] Fetched 2040 reviews (2040 fetched in total)
2021-08-26 23:00:17,661 [INFO] Base - [id:1445504255] Fetched 2560 reviews (2560 fetched in total)
2021-08-26 23:00:22,066 [INFO] Base - [id:1445504255] Fetched 2915 reviews (2915 fetched in to

In [15]:
reviews = my_app.reviews

In [16]:
review_df = pd.DataFrame(reviews)
review_df.to_csv('Appdata(coupang-eats).csv', index=False, encoding = 'utf-8-sig')

In [17]:
review_df.head()

,isEdited,title,userName,developerResponse,review,rating,date
0,False,너무너무너무너무 좋아요!!!!!!!!!!!,니니22,"{'id': 13870810, 'body': '고객님 소중한 피드백 감사합니다. 리...",정말 리뷰 잘 안쓰는데 너무 좋아서 리뷰 남겨요. 원래도 로켓배송을 너무 좋아해서 ...,5,2019-09-11 02:15:15
1,False,잘 쓰고 있었는데...,오류해결좀55467,NaN,앱자체는 좋아 별점 5점으로 리뷰 남겨요\n\n전까진 잘 썼는데 요즘 오류가 너무 ...,5,2020-11-17 06:23:54
2,False,"배달의 민족, 요기요 어플 삭제했어요",안뎌린베,"{'id': 13870664, 'body': '고객님 소중한 피드백 감사합니다. 앞...","배민, 요기요 사용할 때는 2주에 한 번 꼴로 시켜먹을까 말까 했어요. 최소주문금액...",5,2019-08-09 09:50:06
3,False,진짜 쿠팡안되겠네요,홍홍쓰쓰,"{'id': 13870673, 'body': '고객님 소중한 피드백 감사합니다. 앞...",아니 이렇게 하면 남는게 있긴합니까? 떡볶이를 15분만에 받아서 먹었습니다;;;;;...,5,2019-08-14 05:45:29
4,False,역시 로켓배달,테오신,"{'id': 13870705, 'body': '고객님 소중한 피드백 감사합니다. 앞...",강서구에도 쿠팡이츠가 들어 왔네요. 평소 배민을 이용하는데 오늘은 배송비가 무료여서...,5,2019-08-09 11:11:53


In [ ]:
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1478213978/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1478213978/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

In [3]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

In [4]:
res_dict = [] 
for j in range(1,11): 
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1478213978/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1478213978/sortBy=mostRecent/xml' % j
    response = urlopen(url).read() 
    
    xml = xmltodict.parse(response) 
    XmlToJson = json.loads(json.dumps(xml))
    
    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'], 
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']), 
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']), 
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']), 
            'TITLE' : XmlToJson['feed']['entry'][i]['title'], 
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text'] 
        })


In [5]:
res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")

In [6]:
res_df

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2021-08-03 17:41:39,4,0,0,배달원,안녕하세요 쿠팡이츠를 하고있는 배달원입니다 다름이아니라 제가 이글를 올리는 이유는 ...
1,2021-08-02 23:40:59,1,0,0,배달지역,아니 무슨 배달지역이 구분이없네 무슨 왕복 10km를 5천원에 보낸다고 4개 연달아...
2,2021-08-02 10:49:23,1,0,0,안전교육수료,아니 안전교육수료하고 배달1건하면 2만원 준다면서 정산할때는 2만원만 쏙 빼놓고 주...
3,2021-08-01 23:07:43,1,0,0,단가 후려치네,인제는 하다하다 프로모션에 기본단가까지 후려치네...\n왜 어플지도에 나오는 금액이...
4,2021-07-31 13:29:38,1,0,0,잔대가리 굴리는 곰팡이츠,"이벤트 하나하나가 다 낚시, 돈주기 싫어서 평점낚시는 기본으로 깔고 순진한 파트너 ..."
...,...,...,...,...,...,...
485,2019-10-27 08:29:53,1,2,1,픽업시간,중복 콜이 안되면 시간이 돈인데 매장에서10~20분 매번 시다리네요 .....이거 ...
486,2019-10-24 19:37:42,5,1,2,아이폰 드뎌 됩니다,아직 안해봤지만 유투브에서 매장가면 무조건 기다린다는데 정말 저도 홀에 있어봤지만 ...
487,2019-10-23 02:37:11,3,14,3,심각한 문제점,식당업주들은 조리예상시간을 기입해서 배차신청을 하는데 쿠팡이츠는 바로 배달업무배정을...
488,2019-10-15 23:16:56,5,0,0,드디어!,기대되네요 1등!!


In [8]:
res_df.to_csv('Appdata(coupang-eats-rider).csv', header = True, index = False, encoding='utf-8-sig')

In [3]:
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1459484517/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1459484517/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="ko">\n\t\t<id>https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/page=1/id=1459484517/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1459484517/sortBy=mostRecent/xml</id><title>iTunes Store:\xec\x82\xac\xec\x9a\xa9\xec\x9e\x90 \xed\x9b\x84\xea\xb8\xb0</title><updated>2021-08-05T11:12:12-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=kr&amp;id=29141"/><link rel="self" href="https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/page=1/id=1459484517/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1459484517/sortBy=mostRecent/xml"/><link rel="first" href="https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1459484517/sortby=mostrecent/xml?urlDesc=/customerreviews/page=1/id=1459484517/sortby=mostrecent/xml"/><link rel="last" href="https://itu

In [4]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

In [9]:
res_dict = [] 
for j in range(1,3): 
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1459484517/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1459484517/sortBy=mostRecent/xml' % j
    response = urlopen(url).read() 
    
    xml = xmltodict.parse(response) 
    XmlToJson = json.loads(json.dumps(xml))
    
    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'], 
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']), 
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']), 
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']), 
            'TITLE' : XmlToJson['feed']['entry'][i]['title'], 
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text'] 
        })


In [10]:
res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")

In [11]:
res_df

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2021-07-25 18:43:59,1,0,0,갑질,갑질 개 심하면서\n처리도 개떡같고 미안해하지도 않고\n상담원도 어쩌라는식으로 배 ...
1,2021-06-17 23:28:45,1,0,0,엉망진창 ㅋㅋ,어플 별점 총5점에 1몇점인데도 개선의지 1도없는 갑질기업 ㅋㅋㅋ
2,2021-06-14 10:15:42,2,0,0,사용불편,메뉴수정이나 간단한 변경을 위한 작업이 넘 불편합니다. 사용자의 편의성을 높이기 위...
3,2021-06-14 04:21:55,1,0,0,뭔놈에 ㅡㅡ,장사하기 바빠죽겠는데 뭔놈에 소꿉장난하는 놈이 마케팅이랑 프로그램을 맡았는지 다른 ...
4,2021-05-24 19:34:55,1,0,0,ㅋㅋㅋㅋ진짜,아예기본적인것도 똑바로안됭있고\n한두가지여야 뭐라고 말을하지\n상담사랑 담당자랑 말...
...,...,...,...,...,...,...
67,2019-08-06 00:45:07,1,0,0,Ui 무슨일이야;;,"처음 써보려고 앱 깔았는데, 회원가입도 찾을 수 없고, 앱 초기화면도 그냥 다짜고짜..."
68,2019-08-05 22:07:29,5,0,0,라이더용 앱 출시 언제합니까,안드로이드만 출시하고.. 아이폰은 언제해줍니까
69,2019-07-14 02:18:44,5,0,0,앱푸시 문제,메뉴 관리때문에 전 직원 앱을 다 설치 하였는데 안드로이드 폰에서 앱푸시를 꺼 놔도...
70,2019-06-07 21:08:34,5,0,0,로켓배송을 기대합니다!,로켓배송을 기대합니다!


In [12]:
res_df.to_csv('Appdata(coupang-eats-store).csv', header = True, index = False, encoding='utf-8-sig')